In [1]:
!rm *.lock
!rm *.txt
!ls

cf_model.py  fdstests.ipynb  menv	  README.md	     user.py
config.py    filemanager.py  movie.py	  test.db	     wandb
dataset.py   letterboxd.db   __pycache__  testsources.ipynb  web_spider.py
db.ipynb     main.ipynb      q.py	  train.ipynb


In [2]:
# Import the IMDb class from the imdb package
from imdb import IMDb
import requests
import time

import os
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By  # Import the By class
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import import_ipynb
from user import User
from movie import Movie
import db

from faker import Faker
import random

from config import TMDB_API_KEY
from imdb import Cinemagoer
from collections import deque

from datetime import datetime

from filemanager import FileManager
from q import Queue
from web_spider import WebSpider

from googletrans import Translator, LANGUAGES

import threading


import nltk
nltk.download('vader_lexicon')
from nltk.sentiment import SentimentIntensityAnalyzer

import string
import unicodedata


translator = Translator()

print(TMDB_API_KEY)  # Just to verify it's imported correctly; remove in production

month_map = {"Jan": 1, "Feb": 2, "Mar": 3, "Apr": 4, "May": 5, "Jun": 6,
             "Jul": 7, "Aug": 8, "Sep": 9, "Oct": 10, "Nov": 11, "Dec": 12}
import uuid

importing Jupyter notebook from db.ipynb
cf_model.py  fdstests.ipynb  menv	  README.md	     user.py
config.py    filemanager.py  movie.py	  test.db	     wandb
dataset.py   letterboxd.db   __pycache__  testsources.ipynb  web_spider.py
db.ipynb     main.ipynb      q.py	  train.ipynb
e47716bb6a8a0524a757f0b9320f0b29


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/james/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [3]:
INDEPENDENT_FILMS = {
    "Borders",
    "The Nightingale’s Prayer",
    "Divine Carcass",
    "Letter from My Village",
    "The Wedding Party",
    "Skoonheid",
    "Chronicle of the Years of Fire",
    "Yaaba",
    "Yeelen",
    "The Season of Men",
    "The Battle of Algiers",
    "Waiting for Happiness",
    "District 9",
    "Atlantique",
    "I Am Not a Witch",
    "Cairo Station",
    "La Noire de",
    "Touki Bouki",
    "Abouna",
    "Otelo Burning",
    "Across the Niger",
    "The Cursed Ones",
    "Viva Riva!",
    "Of Good Report",
    "From a Whisper",
    "Princess Tyra",
    "Rising Moon",
    "iNumber Number",
    "Run",
    "Vaya",
    "Timbuktu",
    "Abeni",
    "The Amazing Grace",
    "I Sing of a Well",
    "How to Steal 2 Million",
    "Elelwani",
    "Félicité",
    "Battle of the Souls",
    "Eye of the Storm",
    "The Figurine"
    "The Accused",
    "Angel",
    "El Angel",
    "Asako I & II",
    "Before the Frost",
    "Belmonte",
    "Birds of Passage",
    "Black '47",
    "The Black Book",
    "Border",
    "Bulbul Can Sing",
    "The Dive",
    "Donbass",
    "The Factory",
    "Falls Around Her",
    "The Fireflies Are Gone",
    "Florianópolis Dream",
    "The Great Darkened Days",
    "Heart of the World",
    "I Do Not Care If We Go Down in History as Barbarians",
    "Jinpa",
    "Kingsway",
    "Let Me Fall",
    "Look at Me",
    "Minuscule 2: Mandibles from Far Away",
    "The Most Beautiful Couple",
    "Museum",
    "Night/Ext",
    "One Last Deal",
    "The Other Story",
    "Quién te cantará",
    "The Realm",
    "Redemption",
    "Retrospekt",
    "Roads in February",
    "Rosie",
    "Les Salopes, or the Naturally Wanton Pleasure of Skin",
    "Sew the Winter to My Skin",
    "Sibel",
    "Splinters",
    "Stupid Young Heart",
    "Styx",
    "The Sweet Requiem",
    "That Time of Year",
    "Ulysses & Mona",
    "The Vice of Hope",
    "Winter Flies",
    "Working Woman",
    "37 Seconds",
    "Adam",
    "And the Birds Rained Down",
    "Antigone",
    "Arab Blues",
    "Atlantics",
    "Atlantis",
    "Bacurau",
    "Balloon",
    "The Barefoot Emperor",
    "Beanpole",
    "Beneath the Blue Suburban Skies",
    "Blow the Man Down",
    "The Body Remembers When the World Broke Open",
    "Bombay Rose",
    "Castle in the Ground",
    "Chicuarotes",
    "The Climb",
    "Corpus Christi",
    "The County",
    "Dogs Don't Wear Pants",
    "The Father",
    "Flatland",
    "A Girl Missing",
    "Hala",
    "Henry Glassie: Field Work",
    "Incitement",
    "Instinct",
    "The Invisible Life of Eurídice Gusmão",
    "Jallikattu",
    "Knuckle City",
    "La Llorona",
    "The Last Porno Show",
    "The Long Walk",
    "Made in Bangladesh",
    "Mariam",
    "Maria's Paradise",
    "Les Misérables",
    "Nobadi",
    "Our Lady of the Nile",
    "The Perfect Candidate",
    "Red Fields",
    "Resin",
    "So Long, My Son",
    "South Terminal",
    "Spider",
    "A Sun",
    "Synonyms",
    "Tammy's Always Dying",
    "Three Summers",
    "Verdict",
    "White Lie",
    "A White, White Day",
    "The Wild Goose Lake",
    "You Will Die at Twenty",
    "Bandar Band",
    "New Order",
    "Night of the Kings",
    "Preparations to Be Together for an Unknown Period of Time",
    "Quo Vadis, Aida?",
    "Under the Open Sky",
    "7 Prisoners",
    "Are You Lonesome Tonight?",
    "Compartment No. 6",
    "Costa Brava, Lebanon",
    "The Daughter",
    "The Gravedigger's Wife",
    "The Hill Where Lionesses Roar",
    "Întregalde",
    "Jockey",
    "Kicking Blood",
    "La Soga 2",
    "Maria Chapdelaine",
    "Medusa",
    "Murina",
    "Nobody Has to Know",
    "The Odd-Job Men",
    "The Other Tom",
    "Out of Sync",
    "Small Body",
    "Terrorizers",
    "True Things",
    "Unclenching the Fists",
    "Vengeance Is Mine, All Others Pay Cash",
    "The Wheel",
    "Whether the Weather Is Fine",
    "Aftersun",
    "Alam",
    "Amanda",
    "Ashkal",
    "Autobiography",
    "Beyond the Wall",
    "Blind Willow, Sleeping Woman",
    "Bones of Crows",
    "Coyote",
    "Domingo and the Mist",
    "The End of Sex",
    "EO",
    "Falcon Lake",
    "Fixation",
    "Godland",
    "The Happiest Man in the World",
    "The Hotel",
    "The Pack",
    "Life",
    "Living",
    "Love & Mathematics",
    "Love Life",
    "Luxembourg, Luxembourg",
    "The Maiden",
    "Manticore",
    "Muru",
    "My Sailor, My Love",
    "Nightalk",
    "North of Normal",
    "The Origin of Evil",
    "Plan 75",
    "Return to Dust",
    "R.M.N.",
    "So Much Tenderness",
    "Stellar",
    "Stonewalling",
    "The Swearing Jar",
    "The Umbrella Men",
    "Under the Fig Trees",
    "Valeria Is Getting Married",
    "Victim",
    "Vicenta B.",
    "War Sailor",
    "The Water",
    "We Are Still Here",
    "Wildflower",
    "Winter Boy",
    "The Worst Ones",
    "Zwigato",
    "100 Yards",
    "About Dry Grasses",
    "Banel & Adama",
    "The Breaking Ice",
    "Chuck Chuck Baby",
    "City of Wind",
    "Close Your Eyes",
    "Death of a Whistleblower",
    "The Delinquents",
    "Fallen Leaves",
    "The Feeling That the Time for Doing Something Has Passed",
    "Fitting In",
    "Green Border",
    "A Happy Day",
    "Hey, Viktor!",
    "Holiday",
    "Humanist Vampire Seeking Consenting Suicidal Person",
    "I Do Not Come To You By Chance",
    "In Flames",
    "Inshallah a Boy",
    "Irena's Vow",
    "Je’vida",
    "Kanaval",
    "Limbo",
    "Lost Ladies",
    "The Monk and the Gun",
    "Mountains",
    "National Anthem",
    "The Nature of Love",
    "Perfect Days",
    "A Ravaging Wind",
    "The Reeds",
    "A Road to a Village",
    "Robot Dreams",
    "The Settlers",
    "Shadow of Fire",
    "Shayda",
    "Sira",
    "Snow Leopard",
    "Sweet Dreams",
    "The Teachers' Lounge",
    "They Shot the Piano Player",
    "Toll",
    "Upon Open Sky",
    "We Grown Now",
    "Woodland",
    "Your Mother's Son",
    "Otelo Burning",
    "Across the Niger",
    "The Cursed Ones",
    "Viva Riva!",
    "Of Good Report",
    "From a Whisper",
    "Princess Tyra",
    "Rising Moon",
    "iNumber Number",
    "Run",
    "Vaya",
    "Timbuktu",
    "Abeni",
    "The Amazing Grace",
    "I Sing of a Well",
    "How to Steal 2 Million",
    "Elelwani",
    "Félicité",
    "Battle of the Souls",
    "Eye of the Storm",
    "The Figurine"
}

len(INDEPENDENT_FILMS)


270

In [4]:
#common practise to have users rate at least 20-30 reviews each....
TOTAL_MOVIES = len(INDEPENDENT_FILMS)
TOTAL_USERS = 2400
REVIEW_PAGES = 2
DB_NAME = 'letterboxd'

# INDEPENDENT_FILMS = {"Parasite","Amélie","City of God","Pan's Labyrinth","The Lives of Others","A Separation","Y Tu Mamá También","Let the Right One In","Once",
#     "Nightcrawler","Whale Rider","Dogtooth","The Secret in Their Eyes","Oldboy","Blue Valentine","The Lunchbox","Winter's Bone","Moonlight","Hunt for the Wilderpeople",
#     "Animal Kingdom","Mommy","The Farewell","Persepolis","Boyhood","Manchester by the Sea","Memento","Under the Shadow","The Witch","Lady Bird","Tangerine"
# }


#movie['id'],movie['title'],movie['release_date'],movie['director_id'],movie['overview']
#mr['movie_id'],mr['user_id'],mr['content'],mr['ratings'],mr['date']
# movie['id'],movie['title'],movie['release_date'],movie['director_id'],movie['overview'])

In [5]:
def fetch_movie_names(api_key=TMDB_API_KEY, total_movies=10):
    
    base_url = "https://api.themoviedb.org/3/discover/movie"
    movie_count = 0
    page = 1
    while movie_count < total_movies:
        params = {
            "api_key": api_key,
            "language": "en-US",
            "sort_by": "revenue.desc",
            "page": page
        }
        response = requests.get(base_url, params=params)
        if response.status_code == 200:
            data = response.json()
            for movie in data['results']:

                movie_count += 1
                yield movie['title']
                if movie_count >= total_movies:
                    break
        else:
            print(f"Failed to fetch movies: {response.status_code}")
            break
        page += 1
        time.sleep(1)  # Respect the rate limit
        
    # movie_names[:total_movies]



In [6]:
def scrape_letterboxd_movie(movie_name,review_pages_count):

    # Specify the path to chromedriver
    service = Service(executable_path='/home/james/.wdm/drivers/chromedriver/linux64/121.0.6167.85/chromedriver-linux64/chromedriver')
    # Create ChromeOptions object
    chrome_options = webdriver.ChromeOptions()
    # Disable popups
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--disable-gpu')  # Disable GPU accelerati
    chrome_options.add_argument("--disable-popup-blocking")

    # Initialize the WebDriver with the Service object
    driver = webdriver.Chrome(service=service,options = chrome_options)

    # Wait for the page to load (optional, could use more sophisticated waits)
    url = 'https://letterboxd.com/film/'+ movie_name.replace(' ','-').lower()+ '/'
    driver.get(url)

    # Wait for the cookies consent button to be clickable, and then click it
    wait = WebDriverWait(driver, 30)

    movie = {'title':movie_name,'id':str(uuid.uuid4())}  # Initialize the dictionary once, outside the loop
    
    try:
        
        tabbed_content_blocks = driver.find_elements(By.XPATH, "//div[contains(@class, 'tabbed-content-block')]")
    
        # Loop through each 'tabbed-content' div
        for tcb_e in tabbed_content_blocks:
            name = tcb_e.get_attribute('id').split('-')[-1]
            if name == 'cast':
                movie[name] = [ae.get_attribute('textContent') for ae in tcb_e.find_elements(By.TAG_NAME, 'a')]
            elif name == 'crew':
                movie['director'] = [e.get_attribute('innerText') for e in tcb_e.find_elements(By.XPATH, ".//a[starts-with(@href, '/director/')]")]
                movie['director id'] = str(uuid.uuid4())
                movie['writer'] = [e.get_attribute('innerText') for e in tcb_e.find_elements(By.XPATH, ".//a[starts-with(@href, '/writer/')]")]
            elif name == 'details':
                movie['country'] = [e.get_attribute('textContent') for e in tcb_e.find_elements(By.XPATH, ".//a[starts-with(@href, '/films/country/')]")][0]
            elif name == 'genres':
                movie['genres'] = [e.get_attribute('textContent') for e in tcb_e.find_elements(By.XPATH, ".//a[starts-with(@href, '/films/genre/')]")]
            elif name == 'releases':
                movie['release date'] = tcb_e.find_element(By.XPATH, ".//h5[contains(@class, 'date')]").get_attribute('textContent')
        # reviews
        # Wait for the page to load (optional, could use more sophisticated waits)
        url = 'https://letterboxd.com/film/'+ movie_name.replace(' ','-').lower()+ '/reviews/'
        driver.get(url)

        wait = WebDriverWait(driver, 30)
        
        movie['reviews'] = []
        count = 0
        max_count = review_pages_count
        # and driver.find_element(By.XPATH, "//a[contains(@class, 'next')]"
        while count < max_count:

            review_divs = driver.find_elements(By.XPATH, "//div[contains(@class, 'film-detail-content')]")
        
            for rv in review_divs:
               
                rvw = {'movie_id':movie['id']}

                try:
                    rvw['ratings'] = rv.find_element(By.XPATH, ".//span[starts-with(@class, 'rating')]").get_attribute('class').split('-')[-1]
                    # print(rvw['ratings'])
                except:
                    #impute missing values later....
                    rvw['ratings'] = -1.0

                rvw['date'] = rv.find_element(By.XPATH, ".//span[starts-with(@class, 'date')]").get_attribute('textContent')
                rvw['content'] = rv.find_element(By.XPATH, ".//div[starts-with(@class, 'body-text')]").get_attribute('textContent')
                movie['reviews'].append(rvw)
            
            next_link = driver.find_element(By.XPATH, "//a[contains(@class, 'next')]")
            # next_button = driver.find_element(By.XPATH, "//a[contains(@class, 'next')]")
            driver.execute_script("arguments[0].click();", next_link)

            count += 1
            
    except Exception as e:
        # print(e)
        # print(movie_name)
        print(url)
        # print('exception info finished')
        raise
        # print(count)
    finally:
        driver.quit()
        #perhaps consider returning movie here?
    
    return movie


In [7]:
def process_letterboxd_movie(movie):

    #TODO translate to language...
    # print(len(movie['reviews']))
    try:
        date = movie['release date'].strip().split()
        # print(date)
        movie['release date'] = datetime(int(date[2]),month_map[date[1]],int(date[0]))
    except Exception as e:
        # print(e)
        raise

    # what if no reviews?

    for movie_review in movie['reviews']:
        # print(movie_review['date'])
        date = movie_review['date'].strip().replace('  ',' ').split(' ')[-3:]
        # print(date)
        movie_review['date'] = datetime(int(date[2]),month_map[date[1]],int(date[0]))


            
        if movie_review['ratings'] == -1 and movie_review['content']:
            
            #do quick sentiment analysis.
            try:

                detected = translator.detect(movie_review['content'])


                # Only translate if the detected language is not English
                if detected.lang != 'en':

                    movie_review['content'] = translator.translate(movie_review['content'].strip(),dest='en').text
                    
                sia = SentimentIntensityAnalyzer()
                score = sia.polarity_scores(movie_review['content'])
                sentiment = score['compound']  # Compound score
                movie_review['ratings'] = round(((sentiment + 1) / 2),2)
                # print(movie_review['ratings'])
                
            except Exception as e:
                print(e)
                # print(movie_review['content'])
                
        else:                
            
            movie_review['ratings'] = round(float(movie_review['ratings'])/10.0,2)

    return movie
        
    

In [8]:
mov = scrape_letterboxd_movie('star-wars',2)
process_letterboxd_movie(mov)

{'title': 'star-wars',
 'id': '55f9955c-467c-4609-add5-3eed67e957cd',
 'cast': ['Mark Hamill',
  'Harrison Ford',
  'Carrie Fisher',
  'Peter Cushing',
  'Alec Guinness',
  'Anthony Daniels',
  'Kenny Baker',
  'Peter Mayhew',
  'David Prowse',
  'James Earl Jones',
  'Phil Brown',
  'Shelagh Fraser',
  'Jack Purvis',
  'Alex McCrindle',
  'Eddie Byrne',
  'Drewe Henley',
  'Denis Lawson',
  'Garrick Hagon',
  'Jack Klaff',
  'William Hootkins',
  'Angus MacInnes',
  'Jeremy Sinden',
  'Graham Ashley',
  'Don Henderson',
  'Richard LeParmentier',
  'Leslie Schofield',
  'Michael Leader',
  'David Ankrum',
  'Scott Beach',
  'Show All…',
  'Lightning Bear',
  'Jon Berg',
  'Doug Beswick',
  'Paul Blake',
  'Janice Burchette',
  'Ted Burnett',
  'John Chapman',
  'Gilda Cohen',
  'Tim Condren',
  'Barry Copping',
  'Alfie Curtis',
  'Robert Davies',
  'Maria De Aragon',
  'Robert A. Denham',
  'Frazer Diamond',
  'Peter Diamond',
  'Warwick Diamond',
  'Sadie Eden',
  'Kim Falkinburg',
 

In [9]:
def scrape_imdb_movie_reviews(movie_id):

    # Specify the path to chromedriver
    service = Service(executable_path='/home/james/.wdm/drivers/chromedriver/linux64/121.0.6167.85/chromedriver-linux64/chromedriver')
    
    # Create ChromeOptions object
    chrome_options = webdriver.ChromeOptions()
    
    # Disable popups
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--disable-gpu')  # Disable GPU accelerati

    # Initialize the WebDriver with the Service object
    driver = webdriver.Chrome(service=service,options = chrome_options)

    # url = 'https://www.imdb.com/title/tt'+movie_id+'/reviews'
    # print(url)
    # Open a webpage
    driver.get('https://www.imdb.com/title/tt'+movie_id+'/reviews')

    
    # Wait for the page to load (optional, could use more sophisticated waits)
    time.sleep(5)

    # Wait for the cookies consent button to be clickable, and then click it
    wait = WebDriverWait(driver, 30)
    cookies_button = wait.until(EC.element_to_be_clickable((By.XPATH, '//button[text()="Accept"]')))
    cookies_button.click()

    try:
        max_count = 100
        count = 0
        
        while count < max_count:  # Keep clicking the "Load More" button until it's no longer found

            wait = WebDriverWait(driver, 30)
            load_button = wait.until(EC.element_to_be_clickable((By.XPATH, '//button[text()="Load More"]')))
            load_button.click()
            
            count += 1

            elements = driver.find_elements(By.XPATH, "//div[contains(@class, 'lister-item-content')]")


        #NOTE, webspider not handling submodularisation
        #--------------------------------------------------------------------------------------------------------
        review = {}
        
        for element in elements:
            # print(element)
            #return the individual reviews with their respective ratings
            edivs = element.find_elements(By.TAG_NAME, 'div')
            title_div = element.find_elements(By.TAG_NAME, 'a')

            review = {'ratings':None,'check':''}
            review['title'] = title_div[0].text
            review['movie_id'] = movie_id

            # Step 3: Retrieve and print the text for each child element
            details = {0:'ratings',1:'author',2:'content',3:'check'}
            count = 0

            for div in edivs:
                # print(div.get_attribute('outerHTML'))
                if count in [0,1,2,3]:
                    #possible that there is no rating so check forms.
                    if count == 0:
                        #"10/10"
                        try:
    
                            int(div.text.split('/')[0])
                            review[details[count]] = div.text
                        
                        except:
                            
                            print('review has no rating, skipping')
                            break
                            
                    elif count == 1:
                        data = div.text.split()
                        
                        # print(data)
                        if len(data) >= 1:
                            review['author'] = data[0]
                        if len(data) >= 2:
                            review['date'] = ' '.join(data[1:])
                    else:
    
                        review[details[count]] = div.text
    
                    count +=1
    
            if review['check'] == '':
                #no actual review in these cases.
                review['content'] = None
                
            del review['check']
            # print(review)
            if review['ratings'] != None:
                yield review


    except Exception as e:
        print("All content loaded or button not found.")
    
    
    finally:

    # Close the browser
        driver.quit()

In [10]:
# mix of imdb and tmdb
def search_movie_by_name(movie_name, api_key=TMDB_API_KEY):
    """Searches for a movie by name on TMDb, prints the first result, and fetches reviews including ratings."""
    # Search for the movie

    movie = {'title':None,'release_date':None,'genre':'','director':'','actors':[],'ratings':0,'votes':0,'overview':None,'id':None, 'reviews' : []}
    ia = Cinemagoer()
    # ia = IMDb()
    movies = ia.search_movie(movie_name)
    if movies:
    # Get the first movie from the list
        
        mov = movies[0]
        movkeys = mov.keys()
        print(movkeys)
        # Retrieve detailed information about the movie

        movie['title'] = movie_name
        ia.update(mov)
        if 'genre' in movkeys:
            movie['genre'] = ', '.join(mov['genres'])
        if 'year' in movkeys:
            movie['release date'] = datetime(mov['year'],1,1)
        if 'directors' in movkeys:
            movie['director'] = ', '.join([director['name'] for director in mov['directors']])
        #temporary, even if empty string.
        movie['director_id'] = id(movie['director'])
            
        if 'actors' in movkeys:
            movie['actors'] = [cast['name'] for cast in mov['cast']][:5]
        if 'plot' in movkeys:        
            movie['story'] = mov['plot'][0]
        
        if 'ratings' in movkeys and 'votes' in movkeys:
            
            movie['ratings'] = mov.get('rating')
            movie['votes'] = mov.get('votes')

        movie['id'] = mov['imdbID']
        return movie
    #fill in the details

    else:
        print("No movies found.")

In [11]:
#search_movie_by_name('Avengers: Endgame')

In [12]:
def imdb_scrape_function(movie_name):
    try:
        movie = search_movie_by_name(movie_name)
        movie['reviews'] = [rv for rv in scrape_imdb_movie_reviews(movie['id'])]
        return movie
    except:
        raise
        


In [13]:
def letterboxd_scrape_function(movie_name):
    #preprocess
    title_translator = str.maketrans('', '', string.punctuation)
    clean_name = movie_name.translate(title_translator)
    clean_name = unicodedata.normalize('NFKD', clean_name).encode('ascii', 'ignore').decode('ascii')
    # print('clean name = ',clean_name)
    #get movie info, review pages to go through added.
    
    try:
        movie = scrape_letterboxd_movie(clean_name,REVIEW_PAGES)
        movie = process_letterboxd_movie(movie)
        return movie
        
    except:
        print('caught exception in scraping of',movie_name, 'not adding movie to queue')
        raise
        
    

In [14]:
movies_q = Queue()

def run_spider(spider):
    while not FileManager.is_empty('todo.txt'):
        task = spider.get_from_file('todo.txt')
        if task:
            try:
            #returns review
                movie_info = spider.scrape(task)  # Assuming this method now handles the scraping logic
            #check we actually have a movie worth dishing out????
                print(movie_info.keys())
                print(len(movie_info['reviews']))
            # {'title', 'id', 'cast', 'director', 'director id', 'writer', 'country', 'genres', 'release date', 'reviews'}
                #TODO add writers and cast later on...
                check_set = {'genres','country','title','id','release date','reviews','director','director id'}
                if check_set.issubset(set(movie_info.keys())) and movie_info.get('reviews',[]):
                
                    movies_q.enqueue(movie_info)
                    print('q length of movies completed is :',len(movies_q.items))
                    spider.push_to_file(task, 'done.txt')
    
                else:
                    print('movie has missing data:',movie_info)
            except:
                print('could not scrape')
        else:
            break

In [15]:
def generate_users(cc,count):

    for i in range(0,count):
        yield User(Faker(random.choice(cc)).name())

In [16]:
# TODO content filter
# 
# genre and country,.
# reverse the matching, not most suited, not least suited, 

In [17]:
def main():
    
    db.drop_tables(DB_NAME)
    db.create_db(DB_NAME)
    
    #create a set of movie_names
    # movie_names = list({mn for mn in fetch_movie_names(total_movies = TOTAL_MOVIES)})
    movie_names = list(INDEPENDENT_FILMS)
    print(movie_names)
    #clean for suitability... to letterboxd scrape.
    
    fm = FileManager()
    fm.write_file('todo.txt')
    fm.write_file('done.txt')

    for movie_name in movie_names:

        fm.enqueue_to_file(movie_name,'todo.txt')
    

    country_codes = [
        "ar_AA", "az_AZ", "bg_BG", "bs_BA", "cs_CZ", "de_DE", "dk_DK", "el_GR",
        "en_AU", "en_CA", "en_GB", "en_IN", "en_NZ", "en_US", "es_ES", "es_MX",
        "et_EE", "fa_IR", "fi_FI", "fr_FR", "hi_IN", "hr_HR", "hu_HU", "hy_AM",
        "id_ID", "it_IT", "ja_JP", "ka_GE", "ko_KR", "lt_LT", "lv_LV", "ne_NP", 
        "nl_NL", "no_NO", "pl_PL", "pt_BR", "pt_PT", "ro_RO", "ru_RU", "sk_SK",
        "sl_SI", "sv_SE", "tr_TR", "uk_UA", "zh_CN", "zh_TW"
    ]
    
    users = [user for user in generate_users(country_codes,TOTAL_USERS)]
    
    #real work, get the movies themselves and reviews.
    spiders = [WebSpider(file_manager=FileManager, scrape_function=letterboxd_scrape_function, name=f"Spider-{i}") for i in range(10)]
    
    threads = [threading.Thread(target=run_spider, args=(spider,)) for spider in spiders]
    
    for thread in threads:
        thread.start()
    
    for thread in threads:
        thread.join()
    
    import copy
    
    copy_mq = copy.deepcopy(movies_q)
    user_q = Queue(users)
    leave_q = Queue([])
    
    
    while not movies_q.is_empty():
        mov = movies_q.dequeue()
        
        mov_rq = Queue(mov['reviews'])
        
        #shuffle users before handing out
        random.shuffle(user_q.items)
        # print(len(user_q.items))
        while not user_q.is_empty():
            # if there is anything to hand out.
            user = user_q.dequeue()
            # print(user)
            #we need to hand out to them.....
            if not mov_rq.is_empty():
                review = mov_rq.dequeue()
                review['user_id'] = user.user_id
                user.add_review(review)
                # print(user.name,'has new review',review)
            leave_q.enqueue(user)
            # print(len(user_q))
            # print(len(leave_q))

        #done for that subqueue
        user_q = leave_q
        leave_q = Queue([])

    # print(user_q.items[:10])
    # print(copy_mq.is_empty())

    #now do the DB

    while not user_q.is_empty():
        user = user_q.dequeue()
        db.add_user(user,DB_NAME)
        # print(user.reviews)
        for rv in user.reviews:
            db.add_movie_review(rv,DB_NAME)

    
    while not copy_mq.is_empty():

        movie = copy_mq.dequeue()
        # print(movie['release date'])
        db.add_movie(movie,DB_NAME)

In [18]:
main()

SQLite database and table created locally.
['We Are Still Here', 'Close Your Eyes', 'Manticore', 'The Delinquents', 'The Climb', "Maria's Paradise", 'Kanaval', 'Fitting In', 'A Sun', 'Of Good Report', 'The Battle of Algiers', 'The Other Story', 'Minuscule 2: Mandibles from Far Away', 'White Lie', 'Banel & Adama', 'The Other Tom', 'Toll', 'Bombay Rose', 'Three Summers', 'In Flames', 'North of Normal', 'I Am Not a Witch', 'The Last Porno Show', 'Terrorizers', 'Skoonheid', 'Upon Open Sky', 'Instinct', 'Waiting for Happiness', 'Atlantique', 'Fallen Leaves', 'Borders', 'Les Misérables', 'Murina', 'Death of a Whistleblower', 'Flatland', 'Woodland', '7 Prisoners', 'Living', 'Whether the Weather Is Fine', 'Autobiography', 'The Dive', 'Blow the Man Down', 'Battle of the Souls', 'Chuck Chuck Baby', 'Asako I & II', 'Shayda', 'War Sailor', 'The Realm', 'The Long Walk', 'Shadow of Fire', 'Florianópolis Dream', 'Sibel', 'The Vice of Hope', 'Falcon Lake', 'Vaya', 'Museum', 'Winter Flies', 'Nobadi', '

In [19]:
print(db.display_size(DB_NAME))

Users: 2400

Movies: 178

Reviews: 4272
None


In [20]:
print(db.display_data(DB_NAME,5))

Top 5 Users:
thisuser ('db4f2240-d072-4d0e-8fa4-c564062a65e7', 'Dawid Kolanek')
thisuser ('8551c877-0d13-4366-8dce-c389bf27712e', 'Tiit Jakobson')
thisuser ('bd5d359c-904e-4743-a12c-335ef84e7910', 'Cynthia Rodriguez')
thisuser ('2f5cad5a-a5e4-4fa9-9b23-7ff97f07f25b', 'Janine Bell')
thisuser ('7d6a5863-33c2-4f29-b399-f0695471ff73', 'Molnár E. Ibolya')

Top 5 Movies:
('842bcece-f6d5-4765-82a2-a9893d3590d0', 'Of Good Report', '2013-08-09 00:00:00', '6186c4c2-e310-4bb5-9181-68ce80a942a8', None)
('a9b12d4b-b7a1-4aa3-812b-c7acfd402692', 'Marias Paradise', '2019-09-06 00:00:00', '27c86b65-d9e8-4b70-8495-2c037f96a939', None)
('e0a1f173-dcf0-443d-afe2-11942090dec4', 'The Delinquents', '1989-12-21 00:00:00', '46f88959-dd56-4038-8b22-dcd9945d13a7', None)
('07b77009-75e1-490f-83bb-614e8c98e10e', 'We Are Still Here', '2015-06-05 00:00:00', 'e1b6bb6f-4dc5-432c-9e40-c078fc6fa976', None)
('c5b12804-1039-4d65-9caa-85e6c83885f2', 'The Other Story', '2018-10-25 00:00:00', 'b89fbfa8-393f-4039-bd8b-f20e3f5

In [21]:
print(db.display_all_data(DB_NAME))

Users:
thisuser ('db4f2240-d072-4d0e-8fa4-c564062a65e7', 'Dawid Kolanek')
thisuser ('8551c877-0d13-4366-8dce-c389bf27712e', 'Tiit Jakobson')
thisuser ('bd5d359c-904e-4743-a12c-335ef84e7910', 'Cynthia Rodriguez')
thisuser ('2f5cad5a-a5e4-4fa9-9b23-7ff97f07f25b', 'Janine Bell')
thisuser ('7d6a5863-33c2-4f29-b399-f0695471ff73', 'Molnár E. Ibolya')
thisuser ('e5c6045e-ec81-45c5-90b2-931eb2d90e8a', 'Qüdrət Bağırov')
thisuser ('1b8ed9c6-e103-4450-8ed8-459555459766', 'Elizabeth Hughes')
thisuser ('7b3f9e0f-fead-47ec-8505-5b74e2b0e583', 'سلسبيل ميرفاب')
thisuser ('a0a19f42-a68f-4d4c-a599-de0a9cac5a42', 'Linda Bērziņš')
thisuser ('2f429153-c15a-4622-b9e1-ec1377ccf136', 'Tóth K. Anikó')
thisuser ('a8313639-5189-4740-9f9a-cf6cce786a88', '韓佩珊')
thisuser ('12f3fde9-bb8d-48c7-a0b1-270cb6051a6a', 'Stephanie Stephenson')
thisuser ('b4709106-9129-4f9f-9a62-449d61c0ca69', 'عسل معین')
thisuser ('23ca3ec3-429c-4e11-9258-1176c15504ce', 'Nanni Luria')
thisuser ('761f9a9d-6c99-4fed-9bcf-68c2885b687e', 'Minda